In [ ]:
import os
import math
import time
import new_sim

In [ ]:

class initialNode():
    def __init__(self, name, state):
        self.state = state
        self.child = [0,1,2,3,4,5]
        self.visits = 0
        self.score = 0
        self.name = name

    def save(self):
        with open(self.name+".txt", "w") as f:
            f.write(str(self.state)+"/")
            f.write(str(self.child)+"/")
            f.write(str(self.score)+"/")
            f.write(str(self.visits)+"/")
            f.close()

class TreeNode():
    def __init__(self, state, parent, action, name):
        self.state = state
        self.action = action
        self.parent = parent
        self.name = str(self.parent.name)+"-"+str(name)
        self.child = []
        self.visits = 0
        self.score = 0
        self.terminal_type = None
        self.fully_expanded = None
        self.is_terminal = None

    def save(self):
        with open(self.name+".txt", "w") as f:
            f.write(str(self.state)+"/")
            f.write(str(self.child)+"/")
            f.write(str(self.parent.name)+"/")
            f.write(str(self.score)+"/")
            f.write(str(self.visits)+"/")
            f.write(str(self.action)+"/")
            f.write(str(self.fully_expanded)+"/")
            f.write(str(self.is_terminal)+"/")
            f.write(str(self.terminal_type))
            f.close()

In [ ]:
class mcts():

    def __init__(self):
        self.delay = 1
        self.node_file_path = "/"
        self.exploration_constatnt = 1
        
    def get_data(self, node_file):
        with open(node_file, "r") as f:
            node_list = f.split('/')
            state = node_list[0]
            child = node_list[1]
            parent = node_list[2]
            visits = node_list[4]
            score = node_list[3]
            action = node_list[5]
            is_terminal = node_list[7]
            fully_exp = node_list[6]
            terminal_type = node_list[8]
            f.close()
        return score, parent, visits, state, action, is_terminal, fully_exp, terminal_type, child
    
    def update_node(self, node_name, instance, change):
         if str(instance) == "state": instance = 0
         elif str(instance) == "child": instance = 1
         elif str(instance) == "parent": instance = 2
         elif str(instance) == "score": instance = 3
         elif str(instance) == "visits": instance = 4
         elif str(instance) == "action": instance = 5
         elif str(instance) == "fully_expanded": instance = 6
         elif str(instance) == "is_terminal": instance = 7
         elif str(instance) == "terminal_type": instance = 8
         else: print("wtf")
         with open(node_name, "r") as f:
            node_list = f.split('/')
            for i in range(0, len(node_list)): 
                if instance == i:
                    node_list[i] = change
                else:
                    node_list[i] = node_list[i]
                f.write(str(node_list[i]))
                if not node_list[-1] == node_list[i]:
                    f.write(str("/"))
            f.close()

    def expand(self, node):
        actions = new_sim.get_action()
        state = new_sim.get_state()
        for i in range(0, len(actions)):
            action = actions[i]
            name = str(i)
            parent = node.parent
            child_node = TreeNode(state, parent, action, name)
            child_node.save()
            node.child.append(str(child_node.name))    
        node.fully_expanded = True
        node.save()

    def select(self, state, child_nodes = None):
        full_node_list = os.listdir(self.node_file_path)
        node_list = []
        if child_nodes is None:
            for i in full_node_list:
                if str(i).split("txt")[0].split("-") == 1:
                    node_list.append(i)
        elif child_nodes is not None:
            node_list = child_nodes
        desired_nodes = []
        for file in node_list:
            node_path = str(self.node_file_path)+str(file)
            _, _, node_state, _, _, _, _ = self.get_data(node_path)
            if state == node_state:
                desired_nodes.append(node_path)
        if desired_nodes is not []:
            desired_node = self.chose_node(desired_nodes, state)
            return desired_node
        else:
            if child_nodes is not []:
                _, parent, _, _, _, _, _, _, _ = self.get_data(str(self.node_file_path)+str(child_nodes[0]))
                self.expand(parent)
                _, _, _, _, _, _, _, _, child = self.get_data(str(self.node_file_path)+str(parent))
                desired_node = self.select(state, child)
                return desired_node
            else:
                node = initialNode(state)
                self.expand(node)
                node.save()
                _, _, _, _, _, _, _, _, child = self.get_data(str(self.node_file_path)+str(node))
                desired_node = self.select(state, child)
                return desired_node

    def rollout(self, node):
        new_sim.act(node.action)
        time.sleep(self.delay)
        if not self.is_terminal(node):
            if self.fully_expanded(node):
                _, _, _, _, _, _, _, _, children = self.get_data(str(self.node_file_path)+str(node))
                new_node = self.select(new_sim.get_state(), children)
                self.rollout(new_node)
            else:
                self.expand(node)
                new_node = self.select(new_sim.get_state())
            self.rollout(new_node)
        else:
            return new_node
        
    def is_terminal(self, node):
        _, _, state, _, is_terminal, _, _ = self.get_data(node) 
        if is_terminal is None:
            flag = new_sim.is_terminal(state)
            if not flag:
                name = str(node).split(".txt")
                name = str(name).split("-")
                if len(name) == 10:
                    flag = True
            self.update_node(node, is_terminal, flag)
            return flag
        else:
            return is_terminal

    def backprop(self, node):
        #propagate through the fucking nodes
        score, parent, visits, _, _, _, is_terminal, terminal_type, _ = self.get_data(node)
        self.update_node(node, "visits", visits+1)
        self.update_node(node, "score", score+terminal_type)
        while len(str(parent).split(".txt")[0].slpit("-")) != 1 and is_terminal:
            score, _, visits, _, _, _, _, _, _ = self.get_data(parent)
            self.update_node(parent, "visits", visits+1)
            self.update_node(parent, "score", score+terminal_type)
            _, parent, _, _, _, _, _, _, _ = self.get_data(parent)
        score, _, visits, _, _, _, _, _, _ = self.get_data(parent)
        self.update_node(parent, "visits", visits+1)
        self.update_node(parent, "score", score+terminal_type)
        _, parent, _, _, _, _, _, _, _ = self.get_data(parent)

    def chose_node(self, node_list):
        #here you fucking should use the uct func to chose the fucking appropriate action in a given state
        node_list = []
        uct_scores = []
        for i in node_list:
            score, parent, visits, _, _, _, _, _, _ = self.get_data(i)
            _, _, p_visits, _, _, _, _, _, _ = self.get_data(parent)
            uct = score / visits + self.exploration_constant * math.sqrt(math.log(p_visits / (visits+1)))
            uct_scores.append(uct)
        desired_node = node_list(uct_scores.index(max(uct_scores)))
        return desired_node

In [ ]:
import new_sim
import pygame
import cv2 

carlo = new_sim.Carlo()
while 1:
    carlo.run()
    print(carlo.get_state())
pygame.quit()
cv2.destroyAllWindows()

In [ ]:
list = [1,2,3,4,5,6]

for i in list:
    if 2 == i:
        exit
print("done"+str(i))
print(list.index(3))

In [2]:
list = [1,2,3,4,5,6,7]
for i in range(0, len(list)):
    print(i)

0
1
2
3
4
5
6


In [ ]:
a = 0
while not a==5:
    print(a)
    a += 1
print(a)

In [4]:
a = "1.txt"
b = a.split(".txt")[0].split("-")
b.sort()
b = b[-1]
print(b)

1
